In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-ebkw_g5k
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-ebkw_g5k
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4287 sha256=6953986e5f94809db797f8f7c7fe82e504cfdbb903f8c3b3815149b8013ff0b0
  Stored in directory: /tmp/pip-ephem-wheel-cache-khsi8kgl/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
 %%cu
#include <iostream>
#include <cuda.h>

using namespace std;

#define BLOCK_SIZE 2

__global__ void gpuMM(float *A, float *B, float *C, int N)
{
    // Matrix multiplication for NxN matrices C=A*B
    // Each thread computes a single element of C
    int row = blockIdx.y*blockDim.y + threadIdx.y;
    int col = blockIdx.x*blockDim.x + threadIdx.x;

    float sum = 0.f;
    for (int n = 0; n < N; ++n)
        sum += A[row*N+n]*B[n*N+col];

    C[row*N+col] = sum;
}

int main()
{
    int N;
    float K;
    // Perform matrix multiplication C = A*B
    // where A, B and C are NxN matrices
    // Restricted to matrices where N = K*BLOCK_SIZE;
    K = 2;
    N = K*BLOCK_SIZE;

    cout << "Executing Matrix Multiplcation" << endl;
    cout << "Matrix size: " << N << "x" << N << endl;

    // Allocate memory on the host
    float *hA,*hB,*hC;
    size_t bytes = N*N*sizeof(float);
    hA = (float*)malloc(bytes);
    hB = (float*)malloc(bytes);
    hC = (float*)malloc(bytes);
 
    // Allocate memory on the device
    float *dA,*dB,*dC;
    cudaMalloc(&dA,bytes);
    cudaMalloc(&dB,bytes);
    cudaMalloc(&dC,bytes);

    // Initialize matrices on the host
    for (int j=0; j<N; j++){
        for (int i=0; i<N; i++){
           hA[j*N+i] = 2;
           hB[j*N+i] = 4;

        }
    }

    dim3 threadBlock(BLOCK_SIZE,BLOCK_SIZE);
    dim3 grid(K,K);
    cout<<"\nInput Matrix 1 \n";
    for (int row=0; row<N; row++){
            for (int col=0; col<N; col++){

                   cout<<hA[row*col]<<"	";

            }
            cout<<endl;
        }
    cout<<"\nInput Matrix 2 \n";
    for (int row=0; row<N; row++){
            for (int col=0; col<N; col++){

                   cout<<hB[row*col]<<"	";

            }
            cout<<endl;
        }
    // Copy matrices from the host to device
    cudaMemcpy(dA,hA,bytes,cudaMemcpyHostToDevice);
    cudaMemcpy(dB,hB,bytes,cudaMemcpyHostToDevice);

    //Execute the matrix multiplication kernel

    gpuMM<<<grid,threadBlock>>>(dA,dB,dC,N);

   

    // Now copy the GPU result back to CPU
    cudaMemcpy(hC,dC,bytes,cudaMemcpyDeviceToHost);

    // Check the result and make sure it is correct
    cout <<"\n\n\n\n\n Resultant matrix\n\n";
    for (int row=0; row<N; row++){
        for (int col=0; col<N; col++){

               cout<<hC[row*col]<<"	";

        }
        cout<<endl;
    }

    cout << "Finished." << endl;

}



Executing Matrix Multiplcation
Matrix size: 4x4

Input Matrix 1 
2	2	2	2	
2	2	2	2	
2	2	2	2	
2	2	2	2	

Input Matrix 2 
4	4	4	4	
4	4	4	4	
4	4	4	4	
4	4	4	4	





 Resultant matrix

32	32	32	32	
32	32	32	32	
32	32	32	32	
32	32	32	32	
Finished.

